
# General data

Here, we get vital data for the key airports we want to scrape data from:

location (coordinates, country, city), IATA code,

This then allows us, using the wikipedia API to find each airport's corresponding wikipedia link.

For each time range:

The wikipedia page's raw text can be scrapped predictiably for a list of destinations from an airport. We will encode airport destinations using iata code.

We do this for each airport, generating a large csv file of airport-destination pairs for a particular time change.